In [1]:
pwd

'D:\\Proj_OCR'

In [2]:
import cv2
import torch
from PIL import Image
import cv2
import torch
from PIL import Image
import glob
import matplotlib.pyplot as plt

import numpy as np
import matplotlib.image as mpimg
import os
from pyzbar.pyzbar import decode
from pyzbar import pyzbar
import cv2
import glob
from tqdm import tqdm
from barcode import EAN13
from barcode.writer import ImageWriter
import matplotlib.pyplot as plt

import cv2
import torch
from PIL import Image


%matplotlib inline

## Objects:
#1. Label
class Label:
    def __init__(self,  boundPoints = None, labelImage = None, originCoordinate = None):
        self.boundPoints = boundPoints
        self.labelImage = labelImage
        self.originCoordinate = originCoordinate
        self.barcodeReadStatus = False
    
        #magnified 
        self.magnifiedImage = None
        self.magnificationFactor = 1
        
        #binarized image
        self.binarizedImage = None

        #decoded barcode
        self.barcodeList = []
        
        #bounding box
        self.boundingBox = None
        

In [3]:
files = glob.glob('./Raw_Boxes//*.png')
print(len(files))
another_files = glob.glob('./Raw_Boxes//*.jpg')

for file in another_files:
    files.append(file)
    
    
print(len(files))

0
84


In [4]:
#Model loading
model = torch.hub.load('ultralytics/yolov5', 'custom', path='./Models/detector_sitcker.pt')
model.iou = 0.1
print(type(model))

Using cache found in C:\Users\sendr/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2021-12-9 torch 1.8.2+cu111 CUDA:0 (GeForce RTX 3050 Laptop GPU, 4096MiB)

Fusing layers... 
Model Summary: 224 layers, 7053910 parameters, 0 gradients
Adding AutoShape... 


<class 'models.common.AutoShape'>


In [5]:
class ImageFile:
    def __init__(self, image = None):
        self.image = image
        self.results = []
        self.stickers = []

In [6]:
imgs = []  # batch of images

print('1. File Loading...')
for file in tqdm(files):
    imgs.append(ImageFile(cv2.imread(file)))

print('2. Convert image color ')
for i in tqdm(range (len(imgs))):
    imgs[i].image = cv2.cvtColor(imgs[i].image, cv2.COLOR_BGR2RGB)
    
print('3. Extracting bounding boxes.... ')    
for i in tqdm(range (len(imgs))):
    imgs[i].results = model(np.copy(imgs[i].image))
   
for i in tqdm(range (len(imgs))):
    imgs[i].results.save()
    df = imgs[i].results.pandas().xyxy[0]
    
    if (len(df) >0):
        for j in range (len(df)):
            xmin = int(df.iloc[j].xmin)
            ymin = int(df.iloc[j].ymin)
            xmax = int(df.iloc[j].xmax)
            ymax = int(df.iloc[j].ymax)
            imgs[i].stickers.append(imgs[i].image[ymin:ymax,xmin:xmax])
    
#Collect all stickers from all images
sticker_list = []

for img in imgs:
    for sticker in img.stickers:
        sticker_list.append(sticker)

#check before output  
print(len(sticker_list))
print(type(sticker_list[0]))

1. File Loading...


100%|██████████████████████████████████████████████████████████████████████████████████| 84/84 [00:11<00:00,  7.12it/s]


2. Convert image color 


100%|██████████████████████████████████████████████████████████████████████████████████| 84/84 [00:01<00:00, 62.47it/s]


3. Extracting bounding boxes.... 


 46%|██████████████████████████████████████                                            | 39/84 [00:09<00:12,  3.71it/s]Saved 1 image to runs\detect\exp1118
Saved 1 image to runs\detect\exp1119
 50%|█████████████████████████████████████████                                         | 42/84 [00:09<00:07,  5.92it/s]Saved 1 image to runs\detect\exp1121
Saved 1 image to runs\detect\exp1122
 54%|███████████████████████████████████████████▉                                      | 45/84 [00:09<00:05,  7.67it/s]Saved 1 image to runs\detect\exp1124
Saved 1 image to runs\detect\exp1125
 60%|████████████████████████████████████████████████▊                                 | 50/84 [00:10<00:04,  8.20it/s]Saved 1 image to runs\detect\exp1129
Saved 1 image to runs\detect\exp1130


 62%|██████████████████████████████████████████████████▊                               | 52/84 [00:10<00:03,  9.17it/s]Saved 1 image to runs\detect\exp1131
Saved 1 image to runs\detect\exp1132
 64%|████████████████████████████████████████████████████▋                             | 54/84 [00:10<00:03,  9.77it/s]Saved 1 image to runs\detect\exp1133
Saved 1 image to runs\detect\exp1134
 67%|██████████████████████████████████████████████████████▋                           | 56/84 [00:10<00:02, 10.03it/s]Saved 1 image to runs\detect\exp1135
Saved 1 image to runs\detect\exp1136
 69%|████████████████████████████████████████████████████████▌                         | 58/84 [00:10<00:02, 10.06it/s]Saved 1 image to runs\detect\exp1137
Saved 1 image to runs\detect\exp1138
 71%|██████████████████████████████████████████████████████████▌                       | 60/84 [00:11<00:02, 10.54it/s]Saved 1 image to runs\detect\exp1139
Saved 1 image to runs\detect\exp1140
 74%|██████████████████████████████

835
<class 'numpy.ndarray'>


In [10]:
print('4. outputting result...')
random.shuffle(sticker_list)

4. outputting result...


In [11]:
print('5. outputting result...')
i = 0
prefix = ''
for sticker in tqdm(sticker_list):
    
    if (i < 10):
        prefix = '00'
    elif (i < 100):
        prefix = '0'
    elif (i>100):
        prefix = ''
        
    cv2.imwrite('./Raw_Stickers2/sticker2_'+prefix+str(i)+'.jpg',sticker[:,:,::-1])
    i+=1

5. outputting result...


100%|███████████████████████████████████████████████████████████████████████████████| 835/835 [00:06<00:00, 123.38it/s]


In [9]:
import random

mylist = ["apple", "banana", "cherry"]
random.shuffle(mylist)

print(mylist)

['banana', 'apple', 'cherry']
